#### Forecast - Prophet - Facebook:
---

In [ ]:
import numpy as np
import pandas as pd

from fbprophet import Prophet

#### Entrada de dados:
---

In [ ]:
df = pd.read_csv('../input/london-bike-sharing-dataset/london_merged.csv',parse_dates= ['timestamp'])
df.columns = ["ds","y","t1","t2","hum","wind_speed","weather_code","is_holiday","is_weekend","season"]
df.head()

In [ ]:
df = df.assign(mm3 = df.y.rolling(3).mean(),
               sd3 = df.y.rolling(3).std(),
               mm7 = df.y.rolling(7).mean(),
               sd7 = df.y.rolling(7).std())

df.ds = pd.to_datetime(df.ds)

In [ ]:
df = df.fillna(0)

In [ ]:
train = df[df.ds < "2016-12-01"]
test = df[df.ds >= "2016-12-01"]

In [ ]:
# Modelo aditivo prophet:
m = Prophet(seasonality_mode='additive',
            daily_seasonality=True,
            weekly_seasonality=True,
            yearly_seasonality=False)

# Feriados:
m.add_country_holidays(country_name='US')

# Regressores: Dados da base.
m.add_regressor("t1")
m.add_regressor("t2")
m.add_regressor("hum")
m.add_regressor("wind_speed")
m.add_regressor("weather_code")
m.add_regressor("is_holiday")
m.add_regressor("is_weekend")
m.add_regressor("season")

# Regressores: Estatísticas móveis. 
m.add_regressor('mm3')
m.add_regressor('mm7')
m.add_regressor('sd3')
m.add_regressor('sd7')

# Treinamento.
m.fit(train)

In [ ]:
# Previsão
forecast = m.predict(test)

def fun_mape(y,yhat):
    y = y.values
    yhat = yhat.values
    return np.mean(np.abs((y-yhat)/y)).round(4)

print("mape =",(100-fun_mape(test.y,forecast.yhat)*100).round(0))

df = pd.DataFrame(test.y.values,forecast.yhat.values).reset_index().round(0)
df.columns= ["y","yhat"]

In [ ]:
# Gráfico de componetes
plt = m.plot_components(forecast)

In [ ]:
# Avaliação do treinamento:
import matplotlib.pyplot as plt
f, ax = plt.subplots(figsize=(10,4))
test.plot(kind='line', x='ds', y='y', color='blue', label='Realizado', ax=ax)
forecast.plot(kind='line', x='ds', y='yhat', color='red', label='Forecast', ax=ax)
plt.title('Avaliação das curvas de realizado e de teste')
plt.show()